In [ ]:
import cv2
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Ruta de la carpeta donde están organizadas las subcarpetas de acción con los videos
# videos_root_path = '/content/drive/MyDrive/UNIR/TFE/LSC/Con Video/Videos'
videos_root_path = '/content/drive/MyDrive/UNIR/TFE/LSC/Con Video/Videos/Palabras'

# Ruta de la carpeta donde se guardarán los fotogramas organizados por acción
frames_root_dir = '/content/drive/MyDrive/UNIR/TFE/LSC/Con Video/Frames'

# Cantidad de fotogramas por segundo que se desean guardar de cada video
desired_fps = 20

# Crea la carpeta principal para todos los fotogramas si no existe
os.makedirs(frames_root_dir, exist_ok=True)

# Lista de acciones que esperas encontrar en tu estructura de carpetas
actions = ['bancos', 'baños', 'comidas', 'juegosinfantiles', 'parqueaderos']

# Itera sobre cada acción
for action in actions:
    # Ruta de la carpeta donde están los videos de la acción actual
    videos_path = os.path.join(videos_root_path, action)

    # Ruta de la carpeta donde se guardarán los fotogramas de la acción actual
    action_frames_dir = os.path.join(frames_root_dir, action)
    os.makedirs(action_frames_dir, exist_ok=True)

    # Obtiene la lista de todos los videos en la carpeta de acción
    video_files = glob.glob(os.path.join(videos_path, '*.mp4'))

    # Procesa cada video de la acción
    for sequence, video_path in enumerate(video_files, 1):
        # Abre el video con OpenCV
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"No se pudo abrir el video {video_path}")
            continue

        # Obtiene la cantidad total de fotogramas del video original
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Obtiene la tasa de fotogramas original del video
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Obtiene la duración del video en segundos
        video_duration = total_frames / fps

        # Determina la cantidad de fotogramas a extraer
        desired_frame_count = min(int(video_duration * desired_fps), 20) # Total de frames del video

        # Calcula el frame rate ratio para extraer la cantidad deseada de fotogramas
        frame_rate_ratio = max(total_frames / desired_frame_count, 1)

        # Ruta de la carpeta donde se guardarán los fotogramas de este video
        frames_dir = os.path.join(action_frames_dir, f"{sequence}")
        os.makedirs(frames_dir, exist_ok=True)

        print(f"Procesando video {sequence} de la acción {action}, fps original: {fps}, extrayendo {desired_frame_count} fotogramas de duración {video_duration:.2f} segundos")

        frame_count = 0
        saved_frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()

            if not ret:
                break

            # Extrae cada frame basado en la proporción de la tasa de fotogramas
            if frame_count % frame_rate_ratio < 1:
                # Define el nombre del archivo para el fotograma actual
                frame_filename = f"{saved_frame_count}.jpg"
                frame_path = os.path.join(frames_dir, frame_filename)

                # Guarda el fotograma en la carpeta específica
                cv2.imwrite(frame_path, frame)
                saved_frame_count += 1

            frame_count += 1

        # Libera el objeto cap cuando haya terminado de procesar el video
        cap.release()

        # Cierra todas las ventanas que OpenCV haya abierto
        cv2.destroyAllWindows()

NameError: name 'os' is not defined